# Tennis H2H (maybe over/under) Predictions


---


## Scraping Data

Most valuable sources of data will come from flashscore and ATP website.


Import libraries

In [ ]:
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import random
import time
from datetime import datetime, timedelta
import pandas as pd
import json
import requests
import re
import numpy as np
import pickle
from scraping_utils import get_tournament, create_driver, get_live_player_rankings, extract_results_url, get_tournament_results, get_stat_names, get_stats

filterwarnings function to filter or ignore specific warning messages or categories.

In [2]:
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

--- 

## Get Tournament Data


#### Use scaping_utils functions

In [ ]:
                                                              # Launch browser
tourn_archive_url = ["https://www.atptour.com/en/scores/results-archive?year=2025", 
                     "https://www.atptour.com/en/scores/results-archive?year=2024"]     # ATP tournamnet archive
df_tournaments = get_tournament(tourn_archive_url = tourn_archive_url, save_cache=False)

Save the data in csv

In [ ]:

 # -------------------------------------- iMac -------------------------------------- #
# df_tournaments.to_csv('/Users/samueleferrucci/Library/CloudStorage/GoogleDrive-samueleferrucci94@gmail.com/Other computers/My MacBook Air/Coding/Projects/Tennis ML/data/raw/All_Tournaments.csv', sep=',', columns=df_tournaments.columns, index=False)
 # -------------------------------------- - -------------------------------------- #
 # -------------------------------------- Macbook -------------------------------------- #
df_tournaments.to_csv('/Users/samueleferrucci/Documents/Coding/Projects/Tennis ML/data/raw/All_Tournaments.csv', sep=',', columns=df_tournaments.columns, index=False)
 # -------------------------------------- - -------------------------------------- #

We now have a all the tournaments we want to scrape results and statistics for.

---




## Use API to Obtain Top Player Id's


#### Get Top 500 Live Rankings

The API website is https://rapidapi.com/cantagalloedoardo/api/ultimate-tennis1

In [ ]:
df_top_500_players = get_live_player_rankings(500)

200


Save data in CSV

In [ ]:
# -------------------------------------- iMac -------------------------------------- #
# df_top_500_players.to_csv('/Users/samueleferrucci/Library/CloudStorage/GoogleDrive-samueleferrucci94@gmail.com/Other computers/My MacBook Air/Coding/Projects/Tennis ML/data/raw/top_500_players_23_June.csv', sep=',', columns=top_500_players_23_June.columns, index=False)
# -------------------------------------- - -------------------------------------- #
# -------------------------------------- Macbook -------------------------------------- #
df_top_500_players.to_csv('/Users/samueleferrucci/Library/CloudStorage/GoogleDrive-samueleferrucci94@gmail.com/Other computers/My MacBook Air/Coding/Projects/Tennis ML/data/raw/top_500_players_23_June.csv', sep=',', columns=df_top_500_players.columns, index=False)
# -------------------------------------- - -------------------------------------- #

Saved the dataset to CSV.

---


## Tournament Score Results




#### Tournament Results URL

Then I can loop through all the tournaments

In [2]:
 # -------------------------------------- Macbook -------------------------------------- #
df_tournaments = pd.read_csv('/Users/samueleferrucci/Documents/Coding/Projects/Tennis ML/data/raw/All_Tournaments.csv', sep=',')
df_tournaments["end_date"] = pd.to_datetime(df_tournaments["end_date"])
df_top_500_players = pd.read_csv('/Users/samueleferrucci/Documents/Coding/Projects/Tennis ML/data/raw/top_500_players_23_June.csv', sep=',')
 # -------------------------------------- - -------------------------------------- #
 
 # -------------------------------------- iMac -------------------------------------- #
# df_tournaments = pd.read_csv('/Users/samueleferrucci/Library/CloudStorage/GoogleDrive-samueleferrucci94@gmail.com/Other computers/My MacBook Air/Coding/Projects/Tennis ML/data/raw/All_Tournaments.csv', sep=',')
# df_top_500_players = pd.read_csv('/Users/samueleferrucci/Library/CloudStorage/GoogleDrive-samueleferrucci94@gmail.com/Other computers/My MacBook Air/Coding/Projects/Tennis ML/data/raw/top_500_players_23_June.csv', sep=',')
 # -------------------------------------- - -------------------------------------- #

In [3]:
results_url = extract_results_url(df_tournament=df_tournaments)

#### Get the Scores

Call function get_tournament_results 

In [ ]:
df_tournament_results = get_tournament_results(results_url)




### Concat All Results*

Create a dataframe will all results together __IF__ the ```get_tournament_results``` function saved all the tournament data separately

In [7]:
def process_files(tourn_url):
     
    tournament = tourn_url.split("/")[-4]
    year = tourn_url.split("/")[-2] 
    
    df = pd.read_csv(f"/Users/samueleferrucci/Documents/Coding/Projects/Tennis ML/data/raw/{tournament}_{year}_results.csv", sep=',')

    return(df)

    
frames = [process_files(f) for f in results_url]
df_tournament_results = pd.concat(frames)

---


## Get Tournament Stats



Stats URL's

In [2]:
df_tournament_results = pd.read_csv('/Users/samueleferrucci/Documents/Coding/Projects/Tennis ML/data/raw/all_results.csv', sep=',')

In [3]:
stat_url = df_tournament_results['stats_link'].to_list()

### Call Function To Extract Stats

In [ ]:
stats = get_stats(stat_url[64:66])


Need to get GS's which didnt work earlier

In [23]:
GS_stat_url = []
GS = {'580', '520', '540', '560'}

for url in stat_url:
    if isinstance(url, str):  # Only split strings
        tourn_name = url.split("/")[-2]
        if tourn_name in GS:
            GS_stat_url.append(url)

In [27]:
GS_stats = get_stats(GS_stat_url)

Skipping broken or invalid stat page: https://www.atptour.com//en/scores/match-stats/archive/2025/580/ms008 — Error: string index out of range
Skipping broken or invalid stat page: https://www.atptour.com//en/scores/match-stats/archive/2025/580/ms009 — Error: string index out of range
Skipping broken or invalid stat page: https://www.atptour.com//en/scores/match-stats/archive/2025/580/ms010 — Error: string index out of range
Skipping broken or invalid stat page: https://www.atptour.com//en/scores/match-stats/archive/2025/580/ms011 — Error: string index out of range
Skipping broken or invalid stat page: https://www.atptour.com//en/scores/match-stats/archive/2025/580/ms054 — Error: string index out of range
Skipping broken or invalid stat page: https://www.atptour.com//en/scores/match-stats/archive/2025/520/ms016 — Error: string index out of range
Skipping broken or invalid stat page: https://www.atptour.com//en/scores/match-stats/archive/2025/520/ms017 — Error: string index out of range

In [30]:
df_GS_stats = pd.DataFrame.from_dict(GS_stats)

In [32]:
df_GS_stats.to_csv('/Users/samueleferrucci/Documents/Coding/Projects/Tennis ML/data/raw/GS_stats.csv', sep=',', columns=df_GS_stats.columns, index=False)

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/Users/samueleferrucci/Documents/Coding/Projects/Tennis ML/data/clean/merged_matches.csv')